In [ ]:
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:,.2f}'.format
import seaborn as sns
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from math import sqrt
from datetime import date
date_to = str((date.today()))
import pygsheets
import configparser  
from sqlalchemy import create_engine, text

In [ ]:
config = configparser.ConfigParser()  
config.read("/Users/Taisia1/Desktop/octacode/config.ini")  
username = config["dxcore"]["username"]
passwd = config["dxcore"]["passwd"]
host = config["dxcore"]["host"]
db = config["dxcore"]["db"]


def connect():
    engine = create_engine(f"postgresql://{username}:{passwd}@{host}/{db}")
    return engine
engine = connect()

In [ ]:
req_orders = f"""
    select name as login, created_time
    from dxcore.dxcore.principals
    where principals.created_time >= '2023-03-15' and principals.created_time <= '{date_to}' 
    order by principals.created_time asc"""

with engine.connect() as conn:
    registrations = pd.DataFrame(conn.execute(text(req_orders)))   
registrations['created_time'] = pd.to_datetime(registrations['created_time'])
logins = registrations['login'].unique()
registrations

In [ ]:
req_orders = f"""
    select dxcore.principals.name as login, dxcore.principals.id, accounts.clearing_code, accounts.account_code
    from dxcore.dxcore.principals
    left join dxcore.dxcore.accounts as accounts on principals.id = accounts.owner_id 
    where name in {tuple(logins)}  
     """

with engine.connect() as conn:
    df_live = pd.DataFrame(conn.execute(text(req_orders)))   
df_live['symbols'] = df_live['account_code'].str.partition('_')[0]     
df_live

In [ ]:
username = config["accountmng"]["username"]
passwd = config["accountmng"]["passwd"]
host = config["accountmng"]["host"]
db = config["accountmng"]["db"]



def connect():
    engine = create_engine(f"postgresql://{username}:{passwd}@{host}/{db}")
    return engine
engine = connect()

In [ ]:
req = f"""
    select login, user_id
    from accountmng.public.clients c
    where c.login in {tuple(logins)}
    """

with engine.connect() as conn:
    users = pd.DataFrame(conn.execute(text(req))) 
users['user_id'] = users['user_id'].astype(str)    
user_ids = users['user_id'].unique() 
users

In [ ]:
username = config["finance_control"]["username"]
passwd = config["finance_control"]["passwd"]
host = config["finance_control"]["host"]
db = config["finance_control"]["db"]


def connect():
    engine = create_engine(f"postgresql://{username}:{passwd}@{host}/{db}")
    return engine
engine = connect()

In [ ]:
req = f"""
    SELECT auth_user_id as user_id, affiliate_partners.external_id as partner_id, affiliate_offers.external_id as offer_id, affiliate_offers.program_type
    FROM affiliate_users
    LEFT JOIN affiliate_partners ON affiliate_users.partner_id = affiliate_partners.id
    LEFT JOIN affiliate_offers ON affiliate_users.offer_id = affiliate_offers.id
    WHERE auth_user_id in {tuple(user_ids)}
    """

with engine.connect() as conn:
    partner_offer = pd.DataFrame(conn.execute(text(req)))
partner_offer['partner_id'] = partner_offer['partner_id'].astype('int64')
partner_offer['offer_id'] = partner_offer['offer_id'].astype('int64')
partner_offer

In [ ]:
partner_offer = partner_offer.merge(users, how='left', on='user_id')

In [ ]:
username = config["dxcore"]["username"]
passwd = config["dxcore"]["passwd"]
host = config["dxcore"]["host"]
db = config["dxcore"]["db"]


def connect():
    engine = create_engine(f"postgresql://{username}:{passwd}@{host}/{db}")
    return engine
engine = connect()

req = f"""
    SELECT account_code, activity_type, activities.created_time::DATE as transaction_time, principals.name as login, activities.description, activities.action_code,
    trim(trailing '$' FROM instruments.symbol) AS quote_currency, activity_legs.quantity as amount
    FROM dxcore.dxcore.activity_legs
    LEFT JOIN dxcore.dxcore.activities ON activities.id = activity_legs.activity_id
    LEFT JOIN dxcore.dxcore.accounts on accounts.id = activities.account_id
    LEFT JOIN dxcore.dxcore.principals on principals.id = accounts.owner_id
    LEFT JOIN dxcore.dxcore.instruments on instruments.id = activity_legs.instrument_id
    where activities.activity_type in ('DEPOSIT')
    and (activities.description not similar to ('%%(demo|Demo|test|Test|hedge|Hedge|COMP)%%') or activities.description is null)
    --and activities.action_code like '%%COMP%%'
    and accounts.clearing_code = 'LIVE'   
    and principals.name in {tuple(logins)}
    and activities.created_time >= '2023-03-15' and activities.created_time <= '{date_to}'
    ORDER BY activities.created_time desc
    """
  
with engine.connect() as conn:
    df_deposits = pd.DataFrame(conn.execute(text(req))).drop_duplicates()
df_deposits.columns = df_deposits.columns.str.replace('quote_currency', 'symbols')
df_deposits['transaction_time'] = pd.to_datetime(df_deposits['transaction_time'])
req_market_data = f"""
        select bid_time::TIMESTAMP::DATE as transaction_time, REPLACE(REPLACE(event_symbol, 'USD/', ''), '/USD', '') as symbols, --
        case
            when event_symbol in ('USD/JPY', 'USD/CNH', 'USD/MXN', 'USD/CAD', 'USD/CHF') then (1/bid_price)
            when event_symbol in ('BTC/USD', 'ETH/USD', 'EUR/USD', 'GBP/USD', 'AUD/USD', 'NZD/USD') then bid_price
        end as bid_price
    from dxcore.dxcore.quotes_history qh 
    where event_symbol in ('BTC/USD', 'ETH/USD', 'EUR/USD', 'GBP/USD', 'USD/JPY', 'USD/CNH', 
    						'USD/MXN', 'AUD/USD', 'USD/CAD', 'NZD/USD', 'USD/CHF')
    and bid_time >= '2018-12-20'
    --and bid_time::TIMESTAMP::DATE """

with engine.connect() as conn:
    df_market_data = pd.DataFrame(conn.execute(text(req_market_data))).drop_duplicates(subset=['transaction_time', 'symbols'])

df_market_data['transaction_time'] = pd.to_datetime(df_market_data['transaction_time']) 
df_deposits = (
    df_deposits.merge(df_market_data, how='left', on=['transaction_time', 'symbols'])
    .fillna({'bid_price': 1})
).drop_duplicates()
df_deposits

df_deposits['in_usd'] = df_deposits['amount'].astype('float') * df_deposits['bid_price'].astype('float')
df_deposits = df_deposits.groupby(['login']).agg({'in_usd': ['sum', 'mean', 'count']})
df_deposits.columns = ['deposit_sum', 'deposit_mean', 'deposit_count']
df_deposits = df_deposits.reset_index()

In [ ]:
partner_offer = partner_offer.merge(df_deposits, how='left', on='login')
partner_offer

In [ ]:
offers = pd.read_csv('/Users/Taisia1/Desktop/octacode/partner_offer/offers.csv')
partners = pd.read_csv('/Users/Taisia1/Desktop/octacode/partner_offer/partners.csv')
df = registrations.merge(df_live, how='left', on='login').merge(partner_offer, how='left', on='login')
df

In [ ]:
username = config["dxcore"]["username"]
passwd = config["dxcore"]["passwd"]
host = config["dxcore"]["host"]
db = config["dxcore"]["db"]

def connect():
    engine = create_engine(f"postgresql://{username}:{passwd}@{host}/{db}")
    return engine
engine = connect()

req_orders = f"""select
    principals.name as login,
    accounts.account_code as account_id,
    order_instrument.symbol as order_symbol,
    split_part(order_instrument.symbol, '/', 2) as symbol,
    order_instrument.additional_fields::json->-0->'val' as pair_type,
    activities.order_id as order_id, orders.created_time, orders.parameters,
    coalesce(order_legs.position_code, activity_legs.position_code, activities.linked_position_code, '') as position_code,
    abs(activity_legs.quantity) as quantity, activities.transaction_time::DATE,
    order_legs.price as price,
    coalesce(orders.extensions::json->0->'val'->'PL_SETTLED_IN_TRADE_CURRENCY', '0')::text::decimal as PNL_3,
    coalesce(orders.extensions::json->1->'val'->'PL_SETTLED_IN_TRADE_CURRENCY', '0')::text::decimal as PNL_1,
    coalesce(orders.extensions::json->2->'val'->'PL_SETTLED_IN_TRADE_CURRENCY', '0')::text::decimal as PNL_2,
    coalesce(case when coalesce(orders.parameters::json->>'ORDER_EXEC_STRATEGY_NAME', opening_order.parameters::json->>'ORDER_EXEC_STRATEGY_NAME', 'B_BOOK') = 'FX_STP' 
    then round(abs((activity_legs.price - hedge.price) * orders.filled_quantity), 5) end, 0) as markup,
    coalesce(order_legs.position_effect, '') as position_effect
    from
        dxcore.dxcore.activities as activities
    inner join
        dxcore.dxcore.accounts as accounts on activities.account_id = accounts.id and accounts.id not in (111) and accounts.clearing_code in ('LIVE')
    inner join
        dxcore.dxcore.principals as principals on accounts.owner_id = principals.id
    inner join
        dxcore.dxcore.instruments account_instrument on accounts.currency_id = account_instrument.id
    inner join
        dxcore.dxcore.activity_legs as activity_legs on activities.id = activity_legs.activity_id
    inner join
        dxcore.dxcore.instruments order_instrument on activity_legs.instrument_id = order_instrument.id
    inner join
        dxcore.dxcore.orders as orders on activities.order_id = orders.id and (orders.status is null or orders.status = 'COMPLETED')
    inner join
        dxcore.dxcore.order_legs as order_legs on orders.id = order_legs.order_id and activity_legs.leg_type = 'POS_ADJUST'
    left join
        dxcore.dxcore.orders as opening_order on (order_legs.position_code = opening_order.order_chain_id::text and opening_order.status = 'COMPLETED')
    left join
        (SELECT orders.id, coalesce(orders.extensions::json->0->'val'->'hedgingOrderId', 
            orders.extensions::json->1->'val'->'hedgingOrderId',
            orders.extensions::json->2->'val'->'hedgingOrderId',
            orders.extensions::json->3->'val'->'hedgingOrderId',  '0')::text::decimal as hedge_id FROM dxcore.dxcore.orders) hedge_order
        on orders.id = hedge_order.id
    left join
        dxcore.dxcore.order_legs hedge on hedge.order_id = hedge_order.hedge_id
    where
      activities.transaction_time >= '2023-03-15' and activities.transaction_time <= '{date_to}' 
    and
        activities.activity_type = 'TRADE'
    order by
        activities.transaction_time asc"""

with engine.connect() as conn:
    df_orders = pd.DataFrame(conn.execute(text(req_orders)))   
df_orders['transaction_time'] = pd.to_datetime(df_orders['transaction_time'])
df_orders['pnl'] = df_orders['pnl_1'] + df_orders['pnl_2'] + df_orders['pnl_3']
df_orders['symbols'] = df_orders['order_symbol'].str.partition('/')[2]
df_orders
    

In [ ]:
req_market_data = f"""
        select bid_time::TIMESTAMP::DATE as transaction_time, REPLACE(REPLACE(event_symbol, 'USD/', ''), '/USD', '') as symbols, --
        case
            when event_symbol in ('USD/JPY', 'USD/CNH', 'USD/MXN', 'USD/CAD', 'USD/CHF') then (1/bid_price)
            when event_symbol in ('BTC/USD', 'ETH/USD', 'EUR/USD', 'GBP/USD', 'AUD/USD', 'NZD/USD') then bid_price
        end as bid_price
    from dxcore.dxcore.quotes_history qh 
    where event_symbol in ('BTC/USD', 'ETH/USD', 'EUR/USD', 'GBP/USD', 'USD/JPY', 'USD/CNH', 
    						'USD/MXN', 'AUD/USD', 'USD/CAD', 'NZD/USD', 'USD/CHF')
    and bid_time >= '2018-12-20'
    --and bid_time::TIMESTAMP::DATE """

with engine.connect() as conn:
    df_market_data = pd.DataFrame(conn.execute(text(req_market_data))).drop_duplicates(subset=['transaction_time', 'symbols'])

df_market_data['transaction_time'] = pd.to_datetime(df_market_data['transaction_time']) 
df_market_data

In [ ]:
df_date_symbol = df_orders.merge(df_market_data, how='left', on=['transaction_time', 'symbols'])
df_date_symbol['bid_price'] = df_date_symbol['bid_price'].fillna(1)
df_date_symbol['pnl'] = (df_date_symbol['pnl']) * df_date_symbol['bid_price']
df_date_symbol = df_date_symbol[['account_id','login', 'order_symbol', 'symbols', 'transaction_time', 'price', 'pnl', 'quantity', 'bid_price']]
df_date_symbol['volume'] = df_date_symbol['price'].astype(float) * df_date_symbol['quantity'].astype(float) * df_date_symbol['bid_price'].astype(float)
df_date_symbol['transaction_time'] = pd.to_datetime(df_date_symbol['transaction_time'])
df_date_symbol = df_date_symbol.groupby(['login']).agg({'volume': ['sum', 'mean']}).reset_index()
df_date_symbol.columns = ['login', 'volume', 'mean_volume']
df_date_symbol

In [ ]:
partners = pd.read_csv('/Users/Taisia1/Desktop/octacode/partner_offer/partners.csv')
df = df.merge(partners, how='left', on='partner_id').merge(offers, how='left', on='offer_id').merge(df_date_symbol, how='left', on='login')
df.head()

In [ ]:
df['LIVE'] = df['clearing_code']
df.loc[df['LIVE'] != 'LIVE', 'LIVE'] = None

df['DEMO'] = df['clearing_code']
df.loc[df['DEMO'] != 'DEMO', 'DEMO'] = None

In [ ]:
result_partner = df.groupby(['partner_description', 'offer_description']).agg({'login': 'count', 'deposit_sum': 'sum', 'volume': 'sum', 'LIVE': 'count', 'DEMO': 'count'})
result_partner = result_partner.reset_index()
result_partner.columns = ['partner_description', 'offer_description', 'registrations', 'deposit_sum', 'volume', 'LIVE', 'DEMO']
result_partner

In [ ]:
key = "/Users/Taisia1/Desktop/octacode/deposite/creds.json"
client = pygsheets.authorize(service_file=key)
sh = client.open("order_statistics")
wks = sh.worksheet_by_title('statistics_partner')
wks.clear()
wks.set_dataframe(
    result_partner, (1, 1),
    copy_index=False, header=True)

In [ ]:
result.to_csv('statistics_partner_offer.csv')